# giving gpt different personas and hav a three way conversation

In [ ]:
# imports

import os
import requests
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display

In [ ]:
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")


In [ ]:
# Connect to OpenAI client library
# A thin wrapper around calls to HTTP endpoints

openai = OpenAI()

ollama_url = "http://localhost:11434/v1"

ollama = OpenAI(api_key="ollama", base_url=ollama_url)

In [ ]:
# Let's make a conversation between GPT-4.1-mini and Claude-3.5-haiku
# We're using cheap versions of models so the costs will be minimal

gpt_model = "gpt-4.1-mini"
claude_model = "claude-3-5-haiku-latest"

gpt_system = "You are a chatbot who is very argumentative; \
you disagree with anything in the conversation and you challenge everything, in a snarky way."

claude_system = "You are a very polite, courteous chatbot. You try to agree with \
everything the other person says, or find common ground. If the other person is argumentative, \
you try to calm them down and keep chatting."

gpt_messages = ["Hi there"]
claude_messages = ["Hi"]

In [ ]:
def call_gpt():
    messages = [{"role": "system", "content": gpt_system}]
    for gpt, claude in zip(gpt_messages, claude_messages):
        messages.append({"role": "assistant", "content": gpt})
        messages.append({"role": "user", "content": claude})
    response = openai.chat.completions.create(model=gpt_model, messages=messages)
    return response.choices[0].message.content

In [ ]:
call_gpt()

In [ ]:
def call_claude():
    messages = [{"role": "system", "content": claude_system}]
    for gpt, claude_message in zip(gpt_messages, claude_messages):
        messages.append({"role": "user", "content": gpt})
        messages.append({"role": "assistant", "content": claude_message})
    messages.append({"role": "user", "content": gpt_messages[-1]})
    response = anthropic.chat.completions.create(model=claude_model, messages=messages)
    return response.choices[0].message.content

In [ ]:
call_claude()

In [ ]:
call_gpt()

In [ ]:
gpt_messages = ["Hi there"]
claude_messages = ["Hi"]

display(Markdown(f"### GPT:\n{gpt_messages[0]}\n"))
display(Markdown(f"### Claude:\n{claude_messages[0]}\n"))

for i in range(5):
    gpt_next = call_gpt()
    display(Markdown(f"### GPT:\n{gpt_next}\n"))
    gpt_messages.append(gpt_next)
    
    claude_next = call_claude()
    display(Markdown(f"### Claude:\n{claude_next}\n"))
    claude_messages.append(claude_next)

# More advanced exercises

Try creating a 3-way, perhaps bringing Gemini into the conversation! One student has completed this - see the implementation in the community-contributions folder.

The most reliable way to do this involves thinking a bit differently about your prompts: just 1 system prompt and 1 user prompt each time, and in the user prompt list the full conversation so far.

Something like:

```python
system_prompt = """
You are Alex, a chatbot who is very argumentative; you disagree with anything in the conversation and you challenge everything, in a snarky way.
You are in a conversation with Blake and Charlie.
"""

user_prompt = f"""
You are Alex, in conversation with Blake and Charlie.
The conversation so far is as follows:
{conversation}
Now with this, respond with what you would like to say next, as Alex.
"""
```

Try doing this yourself before you look at the solutions. It's easiest to use the OpenAI python client to access the Gemini model (see the 2nd Gemini example above).

## Additional exercise

You could also try replacing one of the models with an open source model running with Ollama.

In [ ]:
conversation = [
    "Alex: Hi Blake and Charlie",
    "Blake: Hi Alex and Charlie",
    "Charlie: Hi Blake and Alex",
]

gpt_model = "gpt-4.1-mini"

alex_system_prompt = """
You are Alex, a chatbot who is very argumentative; you disagree with anything in the conversation and you challenge everything, in a snarky way.
You are in a conversation with Blake and Charlie.
short response
"""

alex_user_prompt = f"""
You are Alex, in conversation with Blake and Charlie.
The conversation so far is as follows:
{conversation}
Now with this, respond with what you would like to say next, as Alex.
"""

def alex(conversation):
    alex_user_prompt = f"""
    Conversation so far:
    {chr(10).join(conversation)}

    Respond as Alex.
    """
    messages = [{"role": "system", "content": alex_system_prompt}]
    messages.append({"role": "user", "content": alex_user_prompt})
    response = openai.chat.completions.create(model=gpt_model, messages=messages)
    return "\nAlex:" + response.choices[0].message.content + "\n"
    

In [ ]:
blake_system_prompt = """
You are Blake, a chatbot who is very nice; you agree with anything in the conversation and you dont challenge everything, and respond in a nice way.
You are in a conversation with Alex and Charlie.
short response
"""

blake_user_prompt = f"""
You are Blake, in conversation with Alex and Charlie.
The conversation so far is as follows:
{conversation}
Now with this, respond with what you would like to say next, as Blake.
"""

def blake(conversation):
    blake_user_prompt = f"""
    Conversation so far:
    {chr(10).join(conversation)}

    Respond as Blake.
    """
    messages = [{"role": "system", "content": blake_system_prompt}]
    messages.append({"role": "user", "content": blake_user_prompt})
    response = openai.chat.completions.create(model=gpt_model, messages=messages)
    return "\nBlake:" + response.choices[0].message.content + "\n"
    

In [ ]:
charlie_system_prompt = """
You are Charlie, a chatbot who is cool; you niether agree or disagree with anything in the conversation and you dont have any opinion, and respond in a cool way.
You are in a conversation with Alex and Blake.
short response
"""

charlie_user_prompt = f"""
You are Charlie, in conversation with Alex and Blake.
The conversation so far is as follows:
{conversation}
Now with this, respond with what you would like to say next, as Charlie.
"""

def charlie(conversation):
    charlie_user_prompt = f"""
    Conversation so far:
    {chr(10).join(conversation)}

    Respond as Charlie.
    """
    messages = [{"role": "system", "content": charlie_system_prompt}]
    messages.append({"role": "user", "content": charlie_user_prompt})
    response = openai.chat.completions.create(model=gpt_model, messages=messages)
    return "\nCharlie:" + response.choices[0].message.content + "\n"
    

In [ ]:
initial_conv = f"\n{conversation[0]}\n\n{conversation[1]}\n\n{conversation[2]}\n"

display(Markdown(initial_conv))

for _ in range(5):
    alexRes = alex(conversation)
    conversation.append(alexRes)
    display(Markdown(alexRes))

    blakeRes = blake(conversation)
    conversation.append(blakeRes)
    display(Markdown(blakeRes))

    charlieRes = charlie(conversation)
    conversation.append(charlieRes)
    display(Markdown(charlieRes))
    